# 3.1: Classify - Soil
### This notebook is to apply a GPR ML model to soil data in the Seattle area.

In [51]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

OHenc = OneHotEncoder()
Lenc = LabelEncoder()
scale_slope = StandardScaler()
scale_rough = StandardScaler()

In [42]:
# Load the training data CSV file compiled in `1-data_access.ipynb`
input_file = '/home/jovyan/Hackweek2024/multi-gpr-soil/data/2-uw_layer1_trainingdata.csv'
training_data = pd.read_csv(input_file)

# training_data_xr = training_data.to_xarray

# Create a GeoDataFrame from the DataFrame
training_data = gpd.GeoDataFrame(training_data, geometry=gpd.points_from_xy(training_data.LONGITUDE, training_data.LATITUDE))

In [43]:
# list(training_data)#.SIMPLE_USCS
training_data

,BOREHOLE_ID,BOREHOLE_NAME,BOREHOLE_TYPE,BOREHOLE_DEPTH_FT,ELEVATION_FT,LATITUDE,LONGITUDE,LAYER_NUMBER,TOP_DEPTH_FT,BOTTOM_DEPTH_FT,USCS,SIMPLE_USCS,LAYER_THICKNESS_FT,geometry,MAPPED_UNIT,SLOPE,ROUGHNESS
0,7138,B-1,Geotechnical,20.3,167.8,47.656719,-122.305728,1,0.0,1.0,SP,S,1.0,POINT (-122.30573 47.65672),Qvt,3.668571,18.199593
1,7143,B-1-92,Geotechnical,48.5,123.2,47.653642,-122.306837,1,0.0,1.0,ML,M,1.0,POINT (-122.30684 47.65364),Qvt,2.222722,10.232103
2,7144,B-2-92,Geotechnical,42.5,122.9,47.653766,-122.306468,1,0.0,1.0,ML,M,1.0,POINT (-122.30647 47.65377),Qvt,1.996012,9.385303
3,7145,B-3-92,Geotechnical,43.0,117.3,47.653256,-122.306638,1,0.0,1.0,ML,M,1.0,POINT (-122.30664 47.65326),Qvt,3.011285,14.187020
4,7146,B-4-92,Geotechnical,43.0,123.4,47.653709,-122.306259,1,0.0,0.5,SM,S,0.5,POINT (-122.30626 47.65371),Qvt,1.996012,9.385303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,143845,MW-1,Geotechnical,42.0,217.4,47.661044,-122.342453,1,0.0,7.5,SM,S,7.5,POINT (-122.34245 47.66104),Qvr,1.594538,10.106370
502,143846,MW-2,Geotechnical,43.0,218.2,47.661138,-122.342516,1,0.0,7.5,SM,S,7.5,POINT (-122.34252 47.66114),Qvr,1.340698,8.735672
503,143847,MW-3,Geotechnical,45.5,219.3,47.661276,-122.342492,1,0.0,18.0,SM,S,18.0,POINT (-122.34249 47.66128),Qvr,1.594538,10.106370
504,143848,MW-4,Geotechnical,43.0,219.1,47.661177,-122.342383,1,0.0,10.0,SM,S,10.0,POINT (-122.34238 47.66118),Qvr,1.594538,10.106370


In [52]:
X = training_data[['MAPPED_UNIT', 'SLOPE', 'ROUGHNESS']]
# Apply transformations and convert to dense array
# # Preprocessing
# preprocessor = ColumnTransformer(
#     transformers=[
#         # ('cat', OneHotEncoder(), ['MAPPED_UNIT']),  # One-hot encode categorical features
#         ('num', StandardScaler(), ['SLOPE', 'ROUGHNESS'])  # Standardize numerical features
#     ])
X['SLOPE'] = scale_slope.fit_transform(pd.DataFrame(X.SLOPE))
X['ROUGHNESS'] = scale_rough.fit_transform(pd.DataFrame(X.ROUGHNESS))
# X = preprocessor.fit_transform(X).toDataFrame()  # Convert to array if sparse
X = pd.get_dummies(X, columns=['MAPPED_UNIT'], dtype=int)

y_1 = training_data['SIMPLE_USCS']
y = Lenc.fit_transform(y_1.values.reshape(-1,1))  #One Hot Encode y

# Test/Train Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


/tmp/ipykernel_115/439154559.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['SLOPE'] = scale_slope.fit_transform(pd.DataFrame(X.SLOPE))
/tmp/ipykernel_115/439154559.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['ROUGHNESS'] = scale_rough.fit_transform(pd.DataFrame(X.ROUGHNESS))
/srv/conda/envs/notebook/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, 

In [74]:
X_train

,SLOPE,ROUGHNESS,MAPPED_UNIT_Qal,MAPPED_UNIT_Ql,MAPPED_UNIT_Qp,MAPPED_UNIT_Qpf,MAPPED_UNIT_Qpo,MAPPED_UNIT_Qva,MAPPED_UNIT_Qvi,MAPPED_UNIT_Qvr,MAPPED_UNIT_Qvt,MAPPED_UNIT_Qw,MAPPED_UNIT_wtr
477,0.063796,0.226540,0,0,0,0,0,0,0,0,1,0,0
15,0.247559,0.132964,0,0,0,0,0,0,0,0,1,0,0
332,0.343192,0.353111,0,0,0,0,0,0,0,0,1,0,0
423,-1.471228,-1.601774,0,0,1,0,0,0,0,0,0,0,0
19,0.203295,0.367111,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,-0.045330,-0.078223,0,0,0,0,0,0,0,0,1,0,0
270,-0.316931,-0.372642,0,0,0,0,0,0,0,0,1,0,0
348,0.395789,0.156398,0,0,0,0,0,0,0,0,1,0,0
435,-0.402328,-0.265178,0,0,0,0,0,0,0,0,1,0,0


In [85]:
guess_l = (2., 1.)  # In general, x and y have different scales
bounds_l = ((1e-2,100.),) * 2  # Same bounds for x and y
kernel = (
    RBF(length_scale=guess_l, length_scale_bounds=bounds_l)
)

gpc = GaussianProcessClassifier()
gpc.fit(X_train, y_train)

GaussianProcessClassifier()

In [86]:
gpc.score(X, y)

0.8003952569169961

In [87]:
gpc

GaussianProcessClassifier()

In [88]:
Zfit = gpc.predict(X_test)

In [89]:
Zfit

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [90]:
Lenc.classes_

array(['C', 'G', 'M', 'O', 'P', 'S'], dtype=object)